In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import save_model
from matplotlib import pyplot as plt

In [3]:
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}


In [4]:
import pandas as pd


In [6]:
path = "/Users/seemanthrajukurapati/Documents/ml projects/music_recommendation/datasets/netfix_cleaned.csv"
model_path = "/Users/seemanthrajukurapati/Documents/ml projects/music_recommendation/models/emotion_detection_model.h5"
img_path = "/Users/seemanthrajukurapati/Documents/ml projects/music_recommendation/test.jpg"

In [8]:
from tensorflow.keras.models import load_model
model = load_model(model_path)

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [9]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the image from file

img = image.load_img(img_path, target_size=(256, 256))  # Adjust the target size as needed

# Convert the image to a numpy array
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Preprocess the image (if needed)
# Example:
# img_array /= 255.0


# Make predictions
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Map class index to class name
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral',  'sad', 'surprise']
class_index_to_name = {i: class_name for i, class_name in enumerate(emotions)}

# Get the predicted class name
predicted_class_name = class_index_to_name[predicted_class_index]

print("Predicted class:", predicted_class_name)


1/1 [==============================] - 0s 187ms/step
Predicted class: neutral


2024-04-30 16:04:09.681292: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [4]:
movie_data = pd.read_csv(path)

# Get all column names
column_names = movie_data.columns

# Print all column names
print("Column Names:")
for column_name in column_names:
    print(column_name)

Column Names:
names
release_year
maturity_rating
duration
description
genre
mood
cast
subtitles
audio


In [5]:
import pandas as pd
import numpy as np  # Import numpy for NaN handling

def print_unique_values(column_name, path):
    data = pd.read_csv(path)
    unique_values = data[column_name].unique()
    unique_values = [str(value).strip() for value in unique_values if not pd.isnull(value)]  # Remove leading/trailing whitespaces
    unique_values = set(unique_values)
    i = 0
    for value in unique_values:
        print(value)
        i += 1
    print("Total unique values:", i)

print_unique_values("mood", path)  # Change "your_data.csv" to the path of your CSV file



Quirky
Swoonworthy
Romantic
Offbeat
Sentimental
Adrenaline Rush
Heartfelt
Understated
Provocative
Feel-Good
Intimate
Scary
Emotional
Gritty
Mind-Bending
Violent
Exciting
Charming
Steamy
Suspenseful
Goofy
Chilling
Witty
Bittersweet
Dark
Raunchy
Inspiring
Ominous
Total unique values: 28


In [11]:
import pandas as pd

def remove_trailing_comma(df, column_name):
    df[column_name] = df[column_name].str.rstrip(',')  # Remove trailing comma from each value in the specified column
    return df

def update_dataset(path):
    data = pd.read_csv(path)
    updated_data = remove_trailing_comma(data, 'mood')
    updated_data.to_csv(path, index=False)  # Save the updated DataFrame back to the CSV file

update_dataset(path)  # Change "your_data.csv" to the path of your CSV file


In [6]:
import pandas as pd

def get_unique_values(column_name, path):
    data = pd.read_csv(path)
    unique_values = data[column_name].unique()
    unique_values = [str(value).strip() for value in unique_values if pd.notnull(value)]  # Remove leading/trailing whitespaces and handle NaN
    unique_values = list(set(unique_values))
    return unique_values

unique_moods = get_unique_values("mood", path)
print("Unique Moods:", unique_moods)


Unique Moods: ['Quirky', 'Swoonworthy', 'Romantic', 'Offbeat', 'Sentimental', 'Adrenaline Rush', 'Heartfelt', 'Understated', 'Provocative', 'Feel-Good', 'Intimate', 'Scary', 'Emotional', 'Gritty', 'Mind-Bending', 'Violent', 'Exciting', 'Charming', 'Steamy', 'Suspenseful', 'Goofy', 'Chilling', 'Witty', 'Bittersweet', 'Dark', 'Raunchy', 'Inspiring', 'Ominous']


In [8]:
emotion_to_mood = {
    'angry': ['Dark', 'Violent', 'Gritty'],
    'disgust': ['Disgust', 'Chilling', 'Gritty'],
    'fear': ['Scary', 'Suspenseful', 'Dark', 'Chilling'],
    'happy': ['Feel-Good', 'Romantic', 'Charming', 'Quirky'],
    'neutral': ['Neutral', 'Understated', 'Offbeat'],
    'sad': ['Sad', 'Bittersweet', 'Sentimental', 'Emotional'],
    'surprise': ['Exciting', 'Mind-Bending', 'Suspenseful', 'Gritty']
}

In [9]:
def find_unique_mood(emotion, mapping):
    if emotion.lower() not in mapping:
        return "Unknown emotion"
    return mapping[emotion.lower()][0]

# Test the function
emotion = 'happy'
unique_mood = find_unique_mood(emotion, emotion_to_mood)
print(f"The unique mood for {emotion} is: {unique_mood}")

The unique mood for happy is: Feel-Good


In [10]:
movie_path = "/Users/seemanthrajukurapati/Documents/ml projects/music_recommendation/datasets/netfix_cleaned.csv"
movie_data = pd.read_csv(movie_path)

In [12]:
def recommend_movie(user_mood):
   
        user_mood = emotion_to_mood[user_mood]
        recommended_movie = movie_data[movie_data['mood'] == user_mood]
        recommended_movie = recommended_movie.sort_values(by='popularity', ascending=False).head(15)
        
        # Convert DataFrame to a list of dictionaries
        recommended_music_list = []
        
        
        return recommended_music_list
   

In [18]:
import pandas as pd

# Assuming you have loaded your dataset into a DataFrame called `movie_data`

def recommend_movies_by_mood(mood, movie_data):
    # Filter movies based on mood
    mood_filtered_movies = movie_data[movie_data['mood'].str.lower() == mood.lower()]
    
    # Sort movies by release year (you can modify the sorting criteria as per your preference)
    mood_filtered_movies = mood_filtered_movies.sort_values(by='release_year', ascending=False)
    
    # Select specific columns for recommendation
    recommended_movies = mood_filtered_movies[['names', 'duration', 'description', 'release_year']]
    
    return recommended_movies.head(5)

# Example usage:
mood = 'Romantic'  # Change this to any mood you want to recommend movies for
recommended_movies = recommend_movies_by_mood(mood, movie_data)

print(f"Recommended movies for '{mood}':")
print(recommended_movies.to_string(index=False))


Recommended movies for 'Romantic':
                names duration                                                                                                                                             description  release_year
      Indoo Ki Jawani   1h 54m A spirited but naive young woman seeks a one-night stand, but after discovering her date’s real identity, things quickly go from casual to complicated.          2020
Jack the Giant Slayer   1h 54m  After he accidentally opens a gateway that allows giants to enter the kingdom, farm boy Jack seeks to undo the damage and rescue a kidnapped princess.          2013
         Jodhaa Akbar   3h 33m   In 16th-century India, what begins as a strategic alliance between a Mughal emperor and a Hindu princess becomes a genuine opportunity for true love.          2008
         Jodhaa Akbar   3h 33m   In 16th-century India, what begins as a strategic alliance between a Mughal emperor and a Hindu princess becomes a genuine opportunity for true 

In [19]:
def recommend_movies_by_mood(mood, movie_data):
    # Filter movies based on mood
    mood_filtered_movies = movie_data[movie_data['mood'].str.lower() == mood.lower()]
    
    # Sort movies by release year (you can modify the sorting criteria as per your preference)
    mood_filtered_movies = mood_filtered_movies.sort_values(by='release_year', ascending=False)
    
    # Select specific columns for recommendation
    recommended_movies = mood_filtered_movies[['names', 'duration', 'description', 'release_year']]
    
    # Convert DataFrame to list of dictionaries
    recommended_movies_list = recommended_movies.to_dict(orient='records')
    
    return recommended_movies_list[:5]  # Return only the first 5 recommended movies

# Example usage:
mood = 'Romantic'  # Change this to any mood you want to recommend movies for
recommended_movies = recommend_movies_by_mood(mood, movie_data)

print(f"Recommended movies for '{mood}':")
print(recommended_movies)

Recommended movies for 'Romantic':
[{'names': 'Indoo Ki Jawani', 'duration': '1h 54m', 'description': 'A spirited but naive young woman seeks a one-night stand, but after discovering her date’s real identity, things quickly go from casual to complicated.', 'release_year': 2020}, {'names': 'Jack the Giant Slayer', 'duration': '1h 54m', 'description': 'After he accidentally opens a gateway that allows giants to enter the kingdom, farm boy Jack seeks to undo the damage and rescue a kidnapped princess.', 'release_year': 2013}, {'names': 'Jodhaa Akbar', 'duration': '3h 33m', 'description': 'In 16th-century India, what begins as a strategic alliance between a Mughal emperor and a Hindu princess becomes a genuine opportunity for true love.', 'release_year': 2008}, {'names': 'Jodhaa Akbar', 'duration': '3h 33m', 'description': 'In 16th-century India, what begins as a strategic alliance between a Mughal emperor and a Hindu princess becomes a genuine opportunity for true love.', 'release_year': 

In [28]:
limited_emotions_to_moods = {
    'angry': ['Dark', 'Violent', 'Gritty'],
    'disgust': ['Disgust', 'Chilling', 'Gritty'],
    'fear': ['Scary', 'Suspenseful', 'Dark', 'Chilling'],
    'happy': ['Feel-Good', 'Romantic', 'Charming', 'Quirky'],
    'neutral': ['Neutral', 'Understated', 'Offbeat'],
    'sad': ['Sad', 'Bittersweet', 'Sentimental', 'Emotional'],
    'surprise': ['Exciting', 'Mind-Bending', 'Suspenseful', 'Gritty']
}


In [29]:
moods_for_emotion = limited_emotions_to_moods.get(emotion.lower(), [])
print(moods_for_emotion)

['Feel-Good', 'Romantic', 'Charming', 'Quirky']


In [35]:
mood_filtered_movies = pd.DataFrame()
for mood in moods_for_emotion:
    mood_filtered_movies = pd.concat([mood_filtered_movies, movie_data[movie_data['mood'].str.lower() == mood.lower()]])
print(mood_filtered_movies)

                                 names  release_year maturity_rating duration   
6                            Double XL          2022        U/A 13+     2h 5m  \
81               The Croods: A New Age          2020         U/A 7+    1h 35m   
86               Lyle, Lyle, Crocodile          2022         U/A 7+    1h 46m   
96                Hotel Transylvania 2          2015         U/A 7+    1h 29m   
100            Space Jam: A New Legacy          2021         U/A 7+    1h 55m   
..                                 ...           ...             ...      ...   
488  Charlie and the Chocolate Factory          2005         U/A 7+    1h 54m   
508   Roald Dahl's Matilda The Musical          2022         U/A 7+    1h 56m   
513              Johnny English Reborn          2011        U/A 13+    1h 41m   
519                        The Witches          2020         U/A 7+    1h 44m   
523                            Goodbye          2022        U/A 13+    2h 21m   

                           

In [31]:
def recommend_movie(emotion, movie_data):
 
       
        
        # Get the corresponding moods for the given emotion
        moods_for_emotion = limited_emotions_to_moods.get(emotion.lower(), [])
    
        
        # Print unique moods present in the dataset
        unique_moods_in_dataset = movie_data['mood'].str.lower().unique()

        
        # Filter movies based on moods
        mood_filtered_movies = movie_data[movie_data['mood'].str.lower().isin(moods_for_emotion)]
  
        
        # Sort movies by release year
        mood_filtered_movies = mood_filtered_movies.sort_values(by='release_year', ascending=False)
        
        # Select specific columns for recommendation
        recommended_movies = mood_filtered_movies[['names', 'duration', 'description', 'release_year']]
        
        # Convert DataFrame to list of dictionaries
        recommended_movies_list = recommended_movies.to_dict(orient='records')
        
        return recommended_movies_list[:5]  # Return only the first 5 recommended movies
  

    


In [32]:
recommended = recommend_movie("happy", movie_data)
print(recommended)

[]
